# NASA EarthData Login (EDL)
### Authentication for data on-prem and in the cloud

> This tutorial assumes you already know the dataset's `short_name` or the dataset's `concept_id`.

In [1]:
from earthaccess import Auth, Store, DataGranules, DataCollections

auth = Auth().login(strategy="netrc")
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

# The Store class will let us download data from NASA directly
store = Store(auth)

You're now authenticated with NASA Earthdata Login
Using token with expiration date: 09/26/2022



When we use `earthaccess` to search for granules we can pass those results to the `Store` instance to download or open the files.
These file objects are serializable `fsspec` file instances than can be used in `xarray` or `dask`.

## Authentication in the cloud

If the collection is a cloud-hosted collection we can print the `summary()` and get the endpouint to the S3 credentials. These S3 credentials are temporary and we can use them with third party libraries such as s3fs, boto3 or awscli.

In [34]:
from pprint import pprint

collection_query = DataCollections()
# This always returns an array
collections = DataCollections().cloud_hosted(True).keyword("SEA SURFACE HEIGHT").get(2)

for collection in collections:
    pprint(collection.summary())

{'cloud-info': {'Region': 'us-west-2',
                'S3BucketAndObjectPrefixNames': ['podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/',
                                                 'podaac-ops-cumulus-public/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/'],
                'S3CredentialsAPIDocumentationURL': 'https://archive.podaac.earthaccess.nasa.gov/s3credentialsREADME',
                'S3CredentialsAPIEndpoint': 'https://archive.podaac.earthaccess.nasa.gov/s3credentials'},
 'concept-id': 'C2036880672-POCLOUD',
 'file-type': "[{'Format': 'netCDF-4', 'FormatType': 'Native'}]",
 'get-data': ['https://cmr.earthaccess.nasa.gov/virtual-directory/collections/C2036880672-POCLOUD',
              'https://search.earthaccess.nasa.gov/search/granules?p=C2036880672-POCLOUD'],
 'short-name': 'SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812',
 'version': '1812'}
{'cloud-info': {'Region': 'us-west-2',
                'S3Bu

In [36]:
# Now we can use the concept_id to get some granule metadata, this is a global collection so there is no need to use spatial filters
query_counts =  DataGranules(auth).concept_id("C2036880672-POCLOUD").hits()
query_counts

1922

In [54]:
# Let's get granules from 2018, note: this particular dataset has data until 2019
query_results =  DataGranules().concept_id("C2036880672-POCLOUD").temporal("2018-01-01", "2018-12-31").debug(True).get()

query_results[0]

Fetching: https://cmr.earthaccess.nasa.gov/search/granules.umm_json?concept_id[]=C2036880672-POCLOUD&temporal[]=2018-01-01T00:00:00Z,2018-12-31T00:00:00Z
Fetching: https://cmr.earthaccess.nasa.gov/search/granules.umm_json?concept_id[]=C2036880672-POCLOUD&temporal[]=2018-01-01T00:00:00Z,2018-12-31T00:00:00Z


Collection: {'Version': '1812', 'ShortName': 'SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': 0.083, 'SouthBoundingCoordinate': -79.917, 'EastBoundingCoordinate': 180, 'NorthBoundingCoordinate': 79.917}, {'WestBoundingCoordinate': -180, 'SouthBoundingCoordinate': -79.917, 'EastBoundingCoordinate': -0.083, 'NorthBoundingCoordinate': 79.917}]}}}
Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2018-01-04T00:00:00.000Z', 'BeginningDateTime': '2018-01-04T00:00:00.000Z'}}
Size(MB): 15.858672142028809
Data: ['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_2018010412.nc']

In [55]:
print(len(query_results))

73


### Working with the URLs directly

If we chose, we can use `earthaccess` to grab the file's URLs and then download them with another library (if we have a `.netrc` file configured with NASA's EDL credentials)
Getting the links to our data is quiet simple with the `data_links()` method on each of the results:

In [57]:
# if our collections has only one link per granule, some times this is not the case
links = [g.data_links() for g in query_results]
links

[['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_2018010412.nc'],
 ['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_2018010912.nc'],
 ['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_2018011412.nc'],
 ['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_2018011912.nc'],
 ['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_2018012412.nc'],
 ['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_201

## Authentication for onprem collections

In [5]:
granules_query = DataGranules()

query_counts =  granules_query.short_name("ATL06").version("005").bounding_box(-25.31,63.23,-11.95,66.65).hits()
query_counts

1383

In [6]:
granule_results = granules_query.short_name("ATL06").version("005").bounding_box(-25.31,63.23,-11.95,66.65).get(10)
# we can get the links to the data 

links = [g.data_links() for g in granule_results]
links

[['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.14/ATL06_20181014051246_02380105_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.14/ATL06_20181014160213_02450103_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.15/ATL06_20181015044707_02530105_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.16/ATL06_20181016164512_02760103_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.17/ATL06_20181017161933_02910103_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.18/ATL06_20181018155352_03060103_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.19/ATL06_20181019043846_03140105_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.21/ATL06_20181021161113_03520103_005_01.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.22/ATL06_20181022045606_03600105_005_01.h5'],
 ['https://n5eil01u.ecs.nsid

In [7]:
# if our collections has only one link per granule, some times this is not the case
links = [g.data_links()[0] for g in granule_results]
links

['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.14/ATL06_20181014051246_02380105_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.14/ATL06_20181014160213_02450103_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.15/ATL06_20181015044707_02530105_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.16/ATL06_20181016164512_02760103_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.17/ATL06_20181017161933_02910103_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.18/ATL06_20181018155352_03060103_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.19/ATL06_20181019043846_03140105_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.21/ATL06_20181021161113_03520103_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2018.10.22/ATL06_20181022045606_03600105_005_01.h5',
 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL

## Downloading granules from a DAAC hosted collection into our local instance

In [8]:
# here we can use the links or the results
granules = store.get(granule_results, "data/onprem")
granules

 Getting 10 granules, approx download size: 0.13 GB


QUEUEING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

File ATL06_20181014051246_02380105_005_01.h5 already downloaded
File ATL06_20181014160213_02450103_005_01.h5 already downloaded
File ATL06_20181015044707_02530105_005_01.h5 already downloaded
File ATL06_20181016164512_02760103_005_01.h5 already downloaded
File ATL06_20181017161933_02910103_005_01.h5 already downloaded
File ATL06_20181018155352_03060103_005_01.h5 already downloaded
File ATL06_20181019043846_03140105_005_01.h5 already downloaded
File ATL06_20181022154534_03670103_005_01.h5 already downloaded
File ATL06_20181021161113_03520103_005_01.h5 already downloaded
File ATL06_20181022045606_03600105_005_01.h5 already downloaded


PROCESSING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/10 [00:00<?, ?it/s]

## Downloading(or moving if we are in the cloud) granules from a cloud hosted collection into our local instance

In [9]:
query_results =  DataGranules().concept_id("C2036880672-POCLOUD").get(10)
# we print the link to our first granule, to make sure it's a cloud hosted collection
query_results[0].data_links()

['https://archive.podaac.earthaccess.nasa.gov/podaac-ops-cumulus-protected/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812/ssh_grids_v1812_1992100212.nc']

In [10]:
# same call! 
granules = store.get(query_results, "data/cloud")
granules

 Getting 10 granules, approx download size: 0.15 GB


QUEUEING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

File ssh_grids_v1812_1992100212.nc already downloaded
File ssh_grids_v1812_1992100712.nc already downloaded
File ssh_grids_v1812_1992101712.nc already downloaded
File ssh_grids_v1812_1992101212.nc already downloaded
File ssh_grids_v1812_1992102212.nc already downloaded
File ssh_grids_v1812_1992110612.nc already downloaded
File ssh_grids_v1812_1992102712.nc already downloaded
File ssh_grids_v1812_1992110112.nc already downloaded
File ssh_grids_v1812_1992111112.nc already downloaded
File ssh_grids_v1812_1992111612.nc already downloaded


PROCESSING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/10 [00:00<?, ?it/s]

## POC: streaming into xarray

We can use `earthaccess` to stream files directly into xarray, this will work well for cases where:

* Data is in NetCDF/HDF5 format
* Data is not big data (>1TB)
* Data is gridded
* Data is chunked using reasonable large sizes(1MB or more)
  * If our files are chunked in small pieces the access time will be orders of magnitude bigger than just downloading the data and accessing it locally.
  
Opening a year of SSH (SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812) data (1.1 GB approx) takes about 5 minutes using streaming when we access out of region(not in AWS)
Note: It seems that using Dask in `LocalCluster` mode is even detrimental when we stream data(`parallel=True`)

In [61]:
%%time 

import xarray as xr


ds = xr.open_mfdataset(store.open(query_results))

ds

 Opening 73 granules, approx size: 1.13 GB


QUEUEING TASKS | :   0%|          | 0/73 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/73 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/73 [00:00<?, ?it/s]

/home/beto/.pyenv/versions/3.9.6/envs/earthaccess-dev/lib/python3.9/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


CPU times: user 22 s, sys: 4.32 s, total: 26.3 s
Wall time: 5min 26s


<xarray.Dataset>
Dimensions:      (Time: 73, Longitude: 2160, nv: 2, Latitude: 960)
Coordinates:
  * Longitude    (Longitude) float32 0.08333 0.25 0.4167 ... 359.6 359.8 359.9
  * Latitude     (Latitude) float32 -79.92 -79.75 -79.58 ... 79.58 79.75 79.92
  * Time         (Time) datetime64[ns] 2018-01-04T12:00:00 ... 2018-12-30T12:...
Dimensions without coordinates: nv
Data variables:
    Lon_bounds   (Time, Longitude, nv) float32 dask.array<chunksize=(1, 2160, 2), meta=np.ndarray>
    Lat_bounds   (Time, Latitude, nv) float32 dask.array<chunksize=(1, 960, 2), meta=np.ndarray>
    Time_bounds  (Time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    SLA          (Time, Longitude, Latitude) float32 dask.array<chunksize=(1, 2160, 960), meta=np.ndarray>
    SLA_ERR      (Time, Longitude, Latitude) float32 dask.array<chunksize=(1, 2160, 960), meta=np.ndarray>
Attributes: (12/13)
    Conventions:            CF-1.6
    ncei_template_version:  NCEI_NetCDF_Grid_Template_v2.0
    Institution:            Jet Propulsion Laboratory
    geospatial_lat_min:     -79.916664
    geospatial_lat_max:     79.916664
    geospatial_lon_min:     0.083333336
    ...                     ...
    time_coverage_start:    2018-01-04
    time_coverage_end:      2018-01-04
    date_created:           2019-02-11T20:51:21.172490
    version_number:         1812
    summary:                Sea level anomaly grids from altimeter data using...
    title:                  Sea Level Anormaly Estimate based on Altimeter Data

In [62]:
ds.SLA

<xarray.DataArray 'SLA' (Time: 73, Longitude: 2160, Latitude: 960)>
dask.array<concatenate, shape=(73, 2160, 960), dtype=float32, chunksize=(1, 2160, 960), chunktype=numpy.ndarray>
Coordinates:
  * Longitude  (Longitude) float32 0.08333 0.25 0.4167 ... 359.6 359.8 359.9
  * Latitude   (Latitude) float32 -79.92 -79.75 -79.58 ... 79.58 79.75 79.92
  * Time       (Time) datetime64[ns] 2018-01-04T12:00:00 ... 2018-12-30T12:00:00
Attributes:
    units:          m
    long_name:      Sea Level Anomaly Estimate
    standard_name:  sea_surface_height_above_sea_level
    alias:          sea_surface_height_above_sea_level